In [30]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from scipy.spatial import Voronoi
import vispy as vp


import tyssue

from tyssue.core.objects import Epithelium
from tyssue.geometry import bulk_geometry as bgeom
from tyssue.core.generation import (data_dicts, make_df,
                                    hexa_grid3d, from_3d_voronoi)

from vispy import app, gloo, visuals, scene
from vispy.geometry import MeshData


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
grid = hexa_grid3d(12, 12, 3)

### Scatter plots of the cell centers

```python
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import seaborn as sns


fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(*grid.T, c='k', marker='o')

ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

plt.show()
```

In [32]:
datasets = from_3d_voronoi(Voronoi(grid))
eptm = Epithelium('start3D', datasets)

gspecs = eptm.set_geom(bgeom)
bgeom.update_all(eptm)
eptm.cell_df.head()

,y,vol,x,is_alive,num_faces,z,area
cell,,,,,,,
0,0.75,0.000000,-0.125,1,0,0.6875,0
1,0.75,0.020833,1.000,1,0,0.5000,0
2,0.75,0.020833,2.000,1,0,0.5000,0
3,0.75,0.020833,3.000,1,0,0.5000,0
4,0.75,0.020833,4.000,1,0,0.5000,0


In [33]:
eptm.je_df.head()

,nz,nx,dx,length,trgt,srce,cell,ny,face,dy,dz,sub_area,sub_vol
je,,,,,,,,,,,,,
9684,0.3750,0.18750,-1.5,1.677051,91,387,0,0.18750,1494,0.0,0.75,0.229640,0.000000
9686,0.3125,0.15625,0.5,0.707107,88,91,0,0.15625,1494,-0.5,0.00,0.191366,0.000000
9688,0.2500,0.12500,1.0,1.118034,383,88,0,0.12500,1494,0.0,-0.50,0.153093,0.000000
9690,0.3125,0.15625,0.0,0.559017,387,383,0,0.15625,1494,0.5,-0.25,0.191366,0.000000
9482,0.1250,0.06250,0.0,0.559017,388,385,1,0.06250,1272,-0.5,0.25,0.076547,0.002604


In [34]:

vertices, faces, cell_mask = eptm.triangular_mesh(eptm.coords)

canvas = scene.SceneCanvas(keys='interactive', show=True)

grid = canvas.central_widget.add_grid()
view = grid.add_view(0, 1)
#view = canvas.central_widget.add_view()
view.camera =  'turntable'
view.camera.aspect = 1

cell_color = vp.color.ColorArray('#ff0000', alpha=0.5)
cell_color = cell_color.rgba.repeat(eptm.Ne, axis=0)

edge_color = vp.color.ColorArray('#00ff00', alpha=1)

view.bgcolor = vp.color.Color('#ffffff')


mesh = vp.scene.visuals.Mesh(vertices=vertices,
                             faces=faces,
                             face_colors=cell_color)

wire_pos = vertices[eptm.Nc:].copy()


wire = vp.scene.visuals.Line(pos=wire_pos, 
                             connect=faces[:, :2]-eptm.Nc,
                             color=edge_color,
                             width=20)

ccenters = vp.scene.visuals.Markers(pos=eptm.cell_df[eptm.coords].values)

#view.add(mesh)
view.add(wire)
view.add(ccenters)
view.camera.set_range()
canvas.show()

#vp_widget = VispyWidget()
#vp_widget.set_canvas(canvas)



app.run()

0

In [27]:
eptm.je_df.head()

,nz,nx,dx,length,trgt,srce,cell,ny,face,dy,dz,sub_area,sub_vol
je,,,,,,,,,,,,,
9684,0.3750,0.18750,-1.5,1.677051,91,387,0,0.18750,1494,0.0,0.75,0.229640,0.000000
9686,0.3125,0.15625,0.5,0.707107,88,91,0,0.15625,1494,-0.5,0.00,0.191366,0.000000
9688,0.2500,0.12500,1.0,1.118034,383,88,0,0.12500,1494,0.0,-0.50,0.153093,0.000000
9690,0.3125,0.15625,0.0,0.559017,387,383,0,0.15625,1494,0.5,-0.25,0.191366,0.000000
9482,0.1250,0.06250,0.0,0.559017,388,385,1,0.06250,1272,-0.5,0.25,0.076547,0.002604
